<a href="https://colab.research.google.com/github/IlyaZutler/Project-3-Berlin-Airbnb-Ratings/blob/main/7%20test%20new%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Model Selection**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

!pip install openpyxl -q
import openpyxl
import pickle

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# for grid
!pip install geopy -q
from geopy.distance import geodesic

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# CatBoost
!pip install catboost -q
from catboost import CatBoostRegressor, Pool
# XGBoost
import xgboost as xgb
from xgboost import XGBRegressor
# RandomForest
from sklearn.ensemble import RandomForestRegressor
# Linear Regression+l1 Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
# AdaBoost
from sklearn.ensemble import AdaBoostRegressor
# LightGBM
import lightgbm as lgb

from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

!pip -qq install category_encoders
from category_encoders import TargetEncoder

# Set display options
%matplotlib inline
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_colwidth', 1000)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 5.3 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
!pip install catboost -q
from catboost import CatBoostRegressor
import lightgbm as lgb
from xgboost import XGBRegressor
!pip -qq install category_encoders
from category_encoders import TargetEncoder
from sklearn.metrics import mean_squared_error, r2_score


In [ ]:
# apartments = pd.read_excel('https://raw.githubusercontent.com/IlyaZutler/Project-3-Berlin-Airbnb-Ratings/main/apartments_5.xlsx', engine='openpyxl')

In [2]:
# import pickle

!wget https://github.com/IlyaZutler/Project-3-Berlin-Airbnb-Ratings/raw/main/apartments5.pkl -q

with open('apartments5.pkl', 'rb') as f:
    apartments = pickle.load(f)

Drop notinformative columns and make Encodin

In [3]:
apartments.drop(columns=['Host Since','First Review', 'Last Review','Reviews', 'Latitude_Bin', 'Longitude_Bin', 'Latitude_Bin2', 'Longitude_Bi2n','Listing ID'], inplace=True)

apartments['Is Superhost'] = apartments['Is Superhost'].replace({'t': 1, 'f': 0}).fillna(0).astype(int)
apartments['Instant Bookable'] = apartments['Instant Bookable'].replace({'t': 1, 'f': 0}).fillna(0).astype(int)
apartments['Is Exact Location'] = apartments['Is Exact Location'].replace({'t': 1, 'f': 0}).fillna(0).astype(int)

apartments['Host Response Time'] = apartments['Host Response Time'].replace(
    {'within an hour':0, 'within a few hours': 1, 'within a day': 2, 'a few days or more': 3 }).fillna(3).astype(int)


In [8]:
apartments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21334 entries, 0 to 21333
Data columns (total 39 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   Host Response Time    21334 non-null  int64   
 1   Host Response Rate    11622 non-null  float64 
 2   Is Superhost          21334 non-null  int64   
 3   neighbourhood         21334 non-null  object  
 4   Neighborhood Group    21334 non-null  object  
 5   Postal Code           21334 non-null  category
 6   Latitude              21334 non-null  float64 
 7   Longitude             21334 non-null  float64 
 8   Is Exact Location     21334 non-null  int64   
 9   Property Type         21334 non-null  object  
 10  Room Type             21334 non-null  object  
 11  Accomodates           21334 non-null  int64   
 12  Bathrooms             21334 non-null  int64   
 13  Bedrooms              21334 non-null  int64   
 14  Beds                  21334 non-null  int64   
 15  Pr

# Программа для всех сразу

In [12]:
# prompt: преобразовать нечисловые столбфы в категориальные

# Select non-numeric columns
non_numeric_columns = apartments.select_dtypes(exclude=['number']).columns

# Convert non-numeric columns to categorical
apartments[non_numeric_columns] = apartments[non_numeric_columns].astype('category')


In [24]:
X = apartments.drop(columns=['Price', 'Price Log'])
y = apartments['Price']

# rare categories to Other
col_obj = X.select_dtypes(exclude='number').columns
for col in col_obj:
    value_counts = X[col].value_counts()
    rare_categories = value_counts[value_counts < 10].index
    X[col] = X[col].replace(rare_categories, 'Other')

# Convert all columns in `col_obj` to string type
X[col_obj] = X[col_obj].astype(str)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=179)



# Определение категориальных и числовых признаков
cat_features = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
num_features = X_train.select_dtypes(include=['number']).columns.tolist()

# Preprocessor для числовых и категориальных данных
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),  # Заполнение пропусков средним значением для числовых данных
            ('scaler', StandardScaler())
        ]), num_features),

        ('cat_onehot', Pipeline([
            ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),  # Заполнение пропусков для категориальных данных
            ('onehot', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'))
        ]), cat_features)
    ])

# Функция для получения имен признаков после One-Hot Encoding
def get_feature_names_onehot(preprocessor, cat_features, num_features):
    # Обучаем препроцессор на тренировочных данных
    preprocessor.fit(X_train)

    # Получаем имена новых категориальных признаков
    ohe_feature_names = preprocessor.named_transformers_['cat_onehot'].named_steps['onehot'].get_feature_names_out(cat_features)

    # Объединяем числовые и категориальные имена признаков
    all_feature_names = np.concatenate([num_features, ohe_feature_names])

    return all_feature_names

# Полный pipeline для One-Hot Encoding
onehot_full_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', Lasso())
])

# Полный pipeline для Target Encoding (обработка категориальных переменных отдельно)
targetenc_pipeline = Pipeline(steps=[
    ('target_enc', TargetEncoder(min_samples_leaf=10, handle_missing='value', handle_unknown='value'))
])

targetenc_full_pipeline = Pipeline(steps=[
    ('preprocessor', ColumnTransformer(
        transformers=[
            ('num', Pipeline([
                ('imputer', SimpleImputer(strategy='mean')),  # Заполнение пропусков средним значением для числовых данных
                ('scaler', StandardScaler())
            ]), num_features),

            ('cat_targetenc', targetenc_pipeline, cat_features)
        ])),
    ('model', Lasso())
])

# Список моделей
models = {
    'Lasso': Lasso(),
    'Random Forest': RandomForestRegressor(random_state=42),
    'CatBoost': CatBoostRegressor(silent=True, random_state=42),
    'LightGBM': lgb.LGBMRegressor(random_state=42),
    'XGBoost': XGBRegressor(random_state=42)
}

# Функция для обучения модели и сбора метрик
def evaluate_model(pipeline, X_train, X_test, y_train, y_test, model_name, feature_names):
    pipeline.fit(X_train, y_train)
    y_train_pred = pipeline.predict(X_train)
    y_test_pred = pipeline.predict(X_test)

    metrics = {
        'Model': model_name,
        'Train MSE': mean_squared_error(y_train, y_train_pred),
        'Test MSE': mean_squared_error(y_test, y_test_pred),
        'Train R2': r2_score(y_train, y_train_pred),
        'Test R2': r2_score(y_test, y_test_pred)
    }

    # Извлечение важности признаков
    if model_name == 'Lasso':
        feature_importances = np.abs(pipeline.named_steps['model'].coef_)
    else:
        feature_importances = pipeline.named_steps['model'].feature_importances_

    feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
    feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False).reset_index(drop=True)

    return metrics, feature_importance_df

# Таблицы для метрик и важностей признаков
metrics_table = pd.DataFrame()
feature_importance_onehot = pd.DataFrame()
feature_importance_targetenc = pd.DataFrame()

# Оценка моделей с One-Hot Encoding
for model_name, model in models.items():
    onehot_full_pipeline.set_params(model=model)
    all_feature_names = get_feature_names_onehot(preprocessor, cat_features, num_features)

    metrics, feature_importances = evaluate_model(onehot_full_pipeline, X_train, X_test, y_train, y_test, model_name, all_feature_names)
    metrics_table = pd.concat([metrics_table, pd.DataFrame([metrics])], ignore_index=True)
    feature_importance_onehot = pd.concat([feature_importance_onehot, feature_importances.rename(columns={'Importance': model_name})], axis=1)

# Оценка моделей с Target Encoding
for model_name, model in models.items():
    targetenc_full_pipeline.set_params(model=model)
    all_feature_names = np.concatenate([num_features, cat_features])

    metrics, feature_importances = evaluate_model(targetenc_full_pipeline, X_train, X_test, y_train, y_test, model_name, all_feature_names)
    metrics_table = pd.concat([metrics_table, pd.DataFrame([metrics])], ignore_index=True)
    feature_importance_targetenc = pd.concat([feature_importance_targetenc, feature_importances.rename(columns={'Importance': model_name})], axis=1)

# Сохранение метрик и важностей признаков в CSV
metrics_table.to_csv('model_metrics.csv', index=False)
feature_importance_onehot.to_csv('feature_importance_onehot.csv', index=False)
feature_importance_targetenc.to_csv('feature_importance_targetenc.csv', index=False)

# Вывод метрик и важностей
print(metrics_table)
print("Feature Importance with One-Hot Encoding:")
print(feature_importance_onehot)
print("Feature Importance with Target Encoding:")
print(feature_importance_targetenc)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004580 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4332
[LightGBM] [Info] Number of data points in the train set: 17067, number of used features: 689
[LightGBM] [Info] Start training from score 59.431242
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002925 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3766
[LightGBM] [Info] Number of data points in the train set: 17067, number of used features: 37
[LightGBM] [Info] Start training from score 59.431242
           Model   Train MSE    Test MSE  Train R2   Test R2
0          Lasso  871.957390  918.951960  0.484580  0.483271
1  Random Forest  103.022987  779.884267  0.939102  0.561469
2 

In [52]:
metrics_table

,Model,Train MSE,Test MSE,Train R2,Test R2
0,Lasso,871.957390,918.951960,0.484580,0.483271
1,Random Forest,103.022987,779.884267,0.939102,0.561469
2,CatBoost,450.767559,708.462029,0.733548,0.601630
3,LightGBM,479.443476,713.722285,0.716598,0.598672
4,XGBoost,364.235382,745.953305,0.784698,0.580549
5,Lasso,785.886984,858.596348,0.535457,0.517209
6,Random Forest,99.084997,753.494087,0.941430,0.576308
7,CatBoost,360.302146,709.450026,0.787023,0.601074
8,LightGBM,442.315090,717.462293,0.738544,0.596569
9,XGBoost,235.544445,769.706425,0.860768,0.567192


In [25]:
feature_importance_onehot

,Feature,Lasso,Feature,Random Forest,Feature,CatBoost,Feature,LightGBM,Feature,XGBoost
0,Room Type_Private room,17.763347,Accomodates,2.508863e-01,Room Type_Private room,15.723867,Distance_from_center,203,Bedrooms,0.040998
1,Accomodates,9.748674,Room Type_Private room,8.609455e-02,Accomodates,12.241314,Longitude,190,Room Type_Private room,0.040596
2,Bedrooms,7.640069,Bedrooms,6.799010e-02,Bedrooms,7.689859,Host Since Years,189,Bathrooms,0.035529
3,neighbourhood_Mitte,4.504089,Bathrooms,3.941442e-02,Bathrooms,5.564266,Latitude,171,Property Type_Hotel,0.035293
4,Guests Included,4.292955,Distance_from_center,3.737426e-02,Guests Included,5.153972,Last Review Years,157,Accomodates,0.029632
...,...,...,...,...,...,...,...,...,...,...
1300,Bin-Bin_113-66,0.000000,Bin-Bin_89-54,5.609496e-07,Bin-Bin_74-72,0.000000,Bin-Bin_121-59,0,Bin-Bin_78-67,0.000000
1301,Bin-Bin_113-65,0.000000,Bin-Bin_80-51,5.604483e-07,Bin-Bin_74-73,0.000000,Bin-Bin_121-58,0,Bin-Bin_78-68,0.000000
1302,Bin-Bin_113-64,0.000000,Bin2-Bin2_58-29,5.025550e-07,Bin-Bin_75-66,0.000000,Bin-Bin_121-57,0,Postal Code_10781.0,0.000000
1303,Bin-Bin_113-63,0.000000,Bin-Bin_88-45,2.867409e-07,Bin-Bin_75-67,0.000000,Bin-Bin_121-56,0,Bin-Bin_78-70,0.000000


In [50]:
feature_importance_onehot2 = feature_importance_onehot.copy()
for i in range(feature_importance_onehot.shape[0]):
    for j in apartments.columns:
        if j in feature_importance_onehot.iloc[i,0]:
            feature_importance_onehot.iloc[i,0] = j
        if j in feature_importance_onehot.iloc[i,2]:
            feature_importance_onehot.iloc[i,2] = j
        if j in feature_importance_onehot.iloc[i,4]:
            feature_importance_onehot.iloc[i,4] = j
        if j in feature_importance_onehot.iloc[i,6]:
            feature_importance_onehot.iloc[i,6] = j
        if j in feature_importance_onehot.iloc[i,8]:
            feature_importance_onehot.iloc[i,8] = j

In [51]:
feature_importance_onehot

,Feature,Lasso,Feature,Random Forest,Feature,CatBoost,Feature,LightGBM,Feature,XGBoost
0,Room Type,17.763347,Accomodates,2.508863e-01,Room Type,15.723867,Distance_from_center,203,Bedrooms,0.040998
1,Accomodates,9.748674,Room Type,8.609455e-02,Accomodates,12.241314,Longitude,190,Room Type,0.040596
2,Bedrooms,7.640069,Bedrooms,6.799010e-02,Bedrooms,7.689859,Host Since Years,189,Bathrooms,0.035529
3,neighbourhood,4.504089,Bathrooms,3.941442e-02,Bathrooms,5.564266,Latitude,171,Property Type,0.035293
4,Guests Included,4.292955,Distance_from_center,3.737426e-02,Guests Included,5.153972,Last Review Years,157,Accomodates,0.029632
...,...,...,...,...,...,...,...,...,...,...
1300,Bin-Bin,0.000000,Bin-Bin,5.609496e-07,Bin-Bin,0.000000,Bin-Bin,0,Bin-Bin,0.000000
1301,Bin-Bin,0.000000,Bin-Bin,5.604483e-07,Bin-Bin,0.000000,Bin-Bin,0,Bin-Bin,0.000000
1302,Bin-Bin,0.000000,Bin2-Bin2,5.025550e-07,Bin-Bin,0.000000,Bin-Bin,0,Postal Code,0.000000
1303,Bin-Bin,0.000000,Bin-Bin,2.867409e-07,Bin-Bin,0.000000,Bin-Bin,0,Bin-Bin,0.000000


In [53]:
X = apartments.drop(columns=['Price', 'Price Log'])
y = apartments['Price']

# rare categories to Other
col_obj = X.select_dtypes(exclude='number').columns
for col in col_obj:
    value_counts = X[col].value_counts()
    rare_categories = value_counts[value_counts < 10].index
    X[col] = X[col].replace(rare_categories, 'Other')

# Convert all columns in `col_obj` to string type
X[col_obj] = X[col_obj].astype(str)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=179)

# Определение категориальных и числовых признаков
cat_features = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
num_features = X_train.select_dtypes(include=['number']).columns.tolist()

# Preprocessor для числовых и категориальных данных
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),  # Заполнение пропусков средним значением для числовых данных
            ('scaler', StandardScaler())
        ]), num_features),

        ('cat_onehot', Pipeline([
            ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),  # Заполнение пропусков для категориальных данных
            ('onehot', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'))
        ]), cat_features)
    ])

# Функция для получения имен признаков после One-Hot Encoding
def get_feature_names_onehot(preprocessor, cat_features, num_features):
    # Обучаем препроцессор на тренировочных данных
    preprocessor.fit(X_train)

    # Получаем имена новых категориальных признаков
    ohe_feature_names = preprocessor.named_transformers_['cat_onehot'].named_steps['onehot'].get_feature_names_out(cat_features)

    # Объединяем числовые и категориальные имена признаков
    all_feature_names = np.concatenate([num_features, ohe_feature_names])

    return all_feature_names

# Полный pipeline для One-Hot Encoding
onehot_full_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', Lasso())
])

# Полный pipeline для Target Encoding (обработка категориальных переменных отдельно)
targetenc_pipeline = Pipeline(steps=[
    ('target_enc', TargetEncoder(min_samples_leaf=10, handle_missing='value', handle_unknown='value'))
])

targetenc_full_pipeline = Pipeline(steps=[
    ('preprocessor', ColumnTransformer(
        transformers=[
            ('num', Pipeline([
                ('imputer', SimpleImputer(strategy='mean')),  # Заполнение пропусков средним значением для числовых данных
                ('scaler', StandardScaler())
            ]), num_features),

            ('cat_targetenc', targetenc_pipeline, cat_features)
        ])),
    ('model', Lasso())
])

# Список моделей
models = {
    'Lasso': Lasso(),
    'Random Forest': RandomForestRegressor(random_state=42),
    'CatBoost': CatBoostRegressor(silent=True, random_state=42),
    'LightGBM': lgb.LGBMRegressor(random_state=42),
    'XGBoost': XGBRegressor(random_state=42)
}

# Функция для обучения модели и сбора метрик
def evaluate_model(pipeline, X_train, X_test, y_train, y_test, model_name, feature_names, encoding_method):
    pipeline.fit(X_train, y_train)
    y_train_pred = pipeline.predict(X_train)
    y_test_pred = pipeline.predict(X_test)

    metrics = {
        'Model': model_name,
        'Encoding Method': encoding_method,
        'Train MSE': mean_squared_error(y_train, y_train_pred),
        'Test MSE': mean_squared_error(y_test, y_test_pred),
        'Train R2': r2_score(y_train, y_train_pred),
        'Test R2': r2_score(y_test, y_test_pred)
    }

    # Извлечение важности признаков
    if model_name == 'Lasso':
        feature_importances = np.abs(pipeline.named_steps['model'].coef_)
    else:
        feature_importances = pipeline.named_steps['model'].feature_importances_

    feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
    feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False).reset_index(drop=True)

    return metrics, feature_importance_df

# Таблицы для метрик и важностей признаков
metrics_table = pd.DataFrame()
feature_importance_onehot = pd.DataFrame()
feature_importance_targetenc = pd.DataFrame()

# Оценка моделей с One-Hot Encoding
for model_name, model in models.items():
    onehot_full_pipeline.set_params(model=model)
    all_feature_names = get_feature_names_onehot(preprocessor, cat_features, num_features)

    metrics, feature_importances = evaluate_model(onehot_full_pipeline, X_train, X_test, y_train, y_test, model_name, all_feature_names, 'One-Hot Encoding')
    metrics_table = pd.concat([metrics_table, pd.DataFrame([metrics])], ignore_index=True)
    feature_importance_onehot = pd.concat([feature_importance_onehot, feature_importances.rename(columns={'Importance': model_name})], axis=1)

# Оценка моделей с Target Encoding
for model_name, model in models.items():
    targetenc_full_pipeline.set_params(model=model)
    all_feature_names = np.concatenate([num_features, cat_features])

    metrics, feature_importances = evaluate_model(targetenc_full_pipeline, X_train, X_test, y_train, y_test, model_name, all_feature_names, 'Target Encoding')
    metrics_table = pd.concat([metrics_table, pd.DataFrame([metrics])], ignore_index=True)
    feature_importance_targetenc = pd.concat([feature_importance_targetenc, feature_importances.rename(columns={'Importance': model_name})], axis=1)

# Сохранение метрик и важностей признаков в CSV
metrics_table.to_csv('model_metrics.csv', index=False)
feature_importance_onehot.to_csv('feature_importance_onehot.csv', index=False)
feature_importance_targetenc.to_csv('feature_importance_targetenc.csv', index=False)

# Вывод метрик и важностей
print(metrics_table)
print("Feature Importance with One-Hot Encoding:")
print(feature_importance_onehot)
print("Feature Importance with Target Encoding:")
print(feature_importance_targetenc)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004236 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4332
[LightGBM] [Info] Number of data points in the train set: 17067, number of used features: 689
[LightGBM] [Info] Start training from score 59.431242
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003039 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3766
[LightGBM] [Info] Number of data points in the train set: 17067, number of used features: 37
[LightGBM] [Info] Start training from score 59.431242
           Model   Encoding Method   Train MSE    Test MSE  Train R2   Test R2
0          Lasso  One-Hot Encoding  871.957390  918.951960  0.484580  0.483271
1  Random Forest  One-Hot E

In [ ]:
X = apartments.drop(columns=['Price', 'Price Log'])
y = apartments['Price']

# rare categories to Other
col_obj = X.select_dtypes(exclude='number').columns
for col in col_obj:
    value_counts = X[col].value_counts()
    rare_categories = value_counts[value_counts < 10].index
    X[col] = X[col].replace(rare_categories, 'Other')

# Convert all columns in `col_obj` to string type
X[col_obj] = X[col_obj].astype(str)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=179)

# Identify categorical and numerical features
cat_features = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
num_features = X_train.select_dtypes(include=['number']).columns.tolist()

# Preprocessor for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        # Pipeline for numerical features: imputing missing values with mean and standard scaling
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ]), num_features),

        # Pipeline for categorical features: imputing missing values with 'missing' and One-Hot Encoding
        ('cat_onehot', Pipeline([
            ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
            ('onehot', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'))
        ]), cat_features)
    ])

# Function to get feature names after One-Hot Encoding
def get_feature_names_onehot(preprocessor, cat_features, num_features):
    # Fit the preprocessor on the training data
    preprocessor.fit(X_train)

    # Get the names of the new categorical features
    ohe_feature_names = preprocessor.named_transformers_['cat_onehot'].named_steps['onehot'].get_feature_names_out(cat_features)

    # Combine numerical and categorical feature names
    all_feature_names = np.concatenate([num_features, ohe_feature_names])

    return all_feature_names

# Full pipeline for One-Hot Encoding
onehot_full_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', Lasso())
])

# Pipeline for Target Encoding (categorical variables are processed separately)
targetenc_pipeline = Pipeline(steps=[
    ('target_enc', TargetEncoder(min_samples_leaf=10, handle_missing='value', handle_unknown='value'))
])

# Full pipeline for Target Encoding
targetenc_full_pipeline = Pipeline(steps=[
    ('preprocessor', ColumnTransformer(
        transformers=[
            # Pipeline for numerical features: imputing missing values with mean and standard scaling
            ('num', Pipeline([
                ('imputer', SimpleImputer(strategy='mean')),
                ('scaler', StandardScaler())
            ]), num_features),

            # Pipeline for categorical features: Target Encoding
            ('cat_targetenc', targetenc_pipeline, cat_features)
        ])),
    ('model', Lasso())
])

# List of models to be evaluated
models = {
    'Lasso': Lasso(),
    'Random Forest': RandomForestRegressor(random_state=42),
    'CatBoost': CatBoostRegressor(silent=True, random_state=42),
    'LightGBM': lgb.LGBMRegressor(random_state=42),
    'XGBoost': XGBRegressor(random_state=42)
}

# Function to train the model and collect metrics
def evaluate_model(pipeline, X_train, X_test, y_train, y_test, model_name, feature_names, encoding_method):
    # Fit the model pipeline
    pipeline.fit(X_train, y_train)

    # Predict on the train and test sets
    y_train_pred = pipeline.predict(X_train)
    y_test_pred = pipeline.predict(X_test)

    # Calculate and store metrics
    metrics = {
        'Model': model_name,
        'Encoding Method': encoding_method,
        'Train MSE': mean_squared_error(y_train, y_train_pred),
        'Test MSE': mean_squared_error(y_test, y_test_pred),
        'Train R2': r2_score(y_train, y_train_pred),
        'Test R2': r2_score(y_test, y_test_pred)
    }

    # Extract feature importances
    if model_name == 'Lasso':
        feature_importances = np.abs(pipeline.named_steps['model'].coef_)
    else:
        feature_importances = pipeline.named_steps['model'].feature_importances_

    # Create a DataFrame for feature importances
    feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
    feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False).reset_index(drop=True)

    return metrics, feature_importance_df

# Tables to store metrics and feature importances
metrics_table = pd.DataFrame()
feature_importance_onehot = pd.DataFrame()
feature_importance_targetenc = pd.DataFrame()

# Evaluate models with One-Hot Encoding
for model_name, model in models.items():
    onehot_full_pipeline.set_params(model=model)
    all_feature_names = get_feature_names_onehot(preprocessor, cat_features, num_features)

    metrics, feature_importances = evaluate_model(onehot_full_pipeline, X_train, X_test, y_train, y_test, model_name, all_feature_names, 'One-Hot Encoding')
    metrics_table = pd.concat([metrics_table, pd.DataFrame([metrics])], ignore_index=True)
    feature_importance_onehot = pd.concat([feature_importance_onehot, feature_importances.rename(columns={'Importance': model_name})], axis=1)

# Evaluate models with Target Encoding
for model_name, model in models.items():
    targetenc_full_pipeline.set_params(model=model)
    all_feature_names = np.concatenate([num_features, cat_features])

    metrics, feature_importances = evaluate_model(targetenc_full_pipeline, X_train, X_test, y_train, y_test, model_name, all_feature_names, 'Target Encoding')
    metrics_table = pd.concat([metrics_table, pd.DataFrame([metrics])], ignore_index=True)
    feature_importance_targetenc = pd.concat([feature_importance_targetenc, feature_importances.rename(columns={'Importance': model_name})], axis=1)

# Save metrics and feature importances to CSV files
metrics_table.to_csv('model_metrics.csv', index=False)
feature_importance_onehot.to_csv('feature_importance_onehot.csv', index=False)
feature_importance_targetenc.to_csv('feature_importance_targetenc.csv', index=False)

# Display metrics and feature importances
print(metrics_table)
print("Feature Importance with One-Hot Encoding:")
print(feature_importance_onehot)
print("Feature Importance with Target Encoding:")
print(feature_importance_targetenc)
